# Программирование на Golang
#### https://stepik.org/course/54403/syllabus


## Ввод \ Вывод

``` Go
Пакет fmt

"fmt" — реализует форматированный ввод-вывод с аналогичными функциями в Cи
// Это принтэфы сканэфы. В большинстве случаев его достаточно. Даже в production, редки случаи когда нужно что-либо остальное 


// Ввод соответственно выглят так

var text string
fmt.Scan(&string) // & амперсанд перед переменной обязательно!


// Ввод в цикле:

var word string
for i:=0; i < 5; i++ {
    fmt.Scan(&word)
    fmt.Println(word) // Печатем, чтобы посмотеть что только. Что ввели.
}
Но минус, как я понимаю, пакета "fmt" в том, вы будете смеятся он медленный (для ввода, хотя с этим от версии к версии языка борются, вроде что-то в 1.17 писали в патчноуте в 1.18 не смотрел). Но из-за удобства я предпочитаю использовать его.

Пакет os

"os" — предоставляет независимый от платформы интерфейс к функциям операционной системы. Т.е. мы можем использовать потоки ввода/вывода. Т.е. мы можем считывать ввод. // Подробнее рассматривается в курсах Linux.

// Из него нам требуется объект os.Stdin, который позволяет считывать данные с консоли.
Пакет buffio

// Как писал выше "fmt" работет медлинно и чтобы это пофиксить используют буфер —
// некоторая промежуточная область в оперативной памяти выделяется, в которой постепенно
// накапливается информация. В подробности не вдаёмся

"buffio" — собственно и реализует буферизованный ввод-вывод.


// В пакете buffio есть два метода 
// Scanner и Reader

// Scanner используются при чтении построчно. Может использовать регулярки! И если не fmt, то используется чаще всего. И наверное важно отметить Что он не копирует считываемую строку, при каждом вызове Scan она становится удаляется.

// Reader помимо чтения, добавляет буферизацию. Возвращает []byte т.е. используется для чтения байтов. И в свою очередь строка всегда будет скопирована.
buffio Scanner 

// Т.о. для считывания строки с помощью сканера:
// Мы создаем новый объект сканнер, аргументом которому передаем потток ввода.
scanner := bufio.NewScanner(os.Stdin)
// Теперь у нас появляется возможность "слушать" ввод:
scanner.Scan() // возвращает true, пока файл не будет прочитан до конца
// И для передачи того что в потоке в переменую мы преабразовываем это дело в текст
s := scanner.Text()


// Аналогичное в цикле
for scanner.Scan() { // возвращает true, пока файл не будет прочитан до конца
    txt := scanner.Text()
}


Вывод с помощью "зеркального" метода на запись
io.WriteString(os.Stdout, txt)
buffio Reader

 

reader := bufio.NewReader(os.Stdin)

for {
    line, -, err := reader.ReadLine() // ReadLine возвращает line []byte, isPrefix bool, err error
    if err == io.EOF {
	    break
	}
    
    txt, _ := string(line) // Конвертируем байты в строку. Эскейпим ошибку, лень обрабатывать.
    // Для числа, мы бы делали так:
    // num, _ := strconv.Atoi(string(line)) 
}

// Или

for {
    s, err := rd.ReadString('\n')
    if err == io.EOF {
        break
    }
    ...
}

// Вывод

writer := bufio.NewWriter(os.Stdout)
w.WriteString(txt) // Записываем строку
w.Flush() // При выполнении методов WriteString(), WriteRune(), WriteByte() и вы не поверите Write();
          // данные вначале накапливаются в буфере, а чтобы сбросить их в источник данных,
          // необходимо вызвать метод Flush().
```

## Работа с файломи 

### io/ioutil
```Go

Пакет io/ioutil предоставляет скромные, но наиболее востребованные возможности по чтению и записи файлов. Вот практически все содержащиеся в пакете функции:

func ReadFile(filename string) ([]byte, error)

func WriteFile(filename string, data []byte, perm os.FileMode) error
Из сигнатуры этих функций все должно быть предельно ясно: функции реализуют чтение и запись файла, данные считываются / записываются в формате байтового среза. Единственное, что может быть не так очевидно, это параметр perm типа os.FileMode – это права доступа к файлу в битовом формате, знакомом всем, кто знаком с операционными системами *nix.

Следующая функция, которую мы можем использовать:

func ReadAll(r io.Reader) ([]byte, error)
Здесь в качестве аргумента передается не имя файла, а объект типа io.Reader, соответственно функция может читать данные из всех объектов, удовлетворяющих указанному интерфейсу: в т.ч. сетевые соединения и буферы:

b := bytes.NewReader([]byte("Данные в объекте io.Reader"))

data, err := ioutil.ReadAll(b)
if err != nil {
	// ...
}

fmt.Printf("%s\n", data) // Данные в объекте io.Reader
Последняя функция позволяет получить информацию о содержании директории:

func ReadDir(dirname string) ([]os.FileInfo, error)
Не думаю, что в этом вопросе могут возникнуть какие-то проблемы, поэтому просто приведу ряд примеров:
 

dataForFile := []byte("Тестовая строка, предназначенная для записи в файл")

// Создаем новый файл и записываем в него данные dataForFile
if err := ioutil.WriteFile("test.txt", dataForFile, 0600); err != nil {
	...
}

// Читаем данные из того же файла
dataFromFile, err := ioutil.ReadFile("test.txt")
if err != nil {
	...
}

// Сравниваем исходные данные с записанными в файл и прочитанными из него
fmt.Printf("dataForFile == dataFromFile: %v\n", bytes.Equal(dataFromFile, dataForFile))

// Изучаем содержимое директории
filesFromDir, err := ioutil.ReadDir(".")
if err != nil {
	...
}

for _, file := range filesFromDir {
	// Проходим по всем найденным файлам и печатаем их имя и размер
	fmt.Printf("name: %s, size: %d\n", file.Name(), file.Size())
}

// Output:
// dataForFile == dataFromFile: true
// name: main.go, size: 727
// name: test.txt, size: 93

```

### os

```Go

Следующий пакет, который мы просто обязаны рассмотреть, это пакет os. Он содержит огромный набор высокоуровневых инструментов для работы с файлами (и не только).

Для начала мы должны отметить, что рассматриваемый пакет сам не предоставляет удобных инструментов для чтения и записи в файл, хотя теоретически чтение и запись возможны. Центром пакета является объект os.File, который реализует ряд интерфейсов, в т.ч. Reader и Writer. Таким образом для удобной работы с данным типом нам может потребоваться что-то еще, например для чтения всего файла может быть использована функция ReadAll из ранее рассмотренного пакета io/ioutil. Для создания рассматриваемого объекта мы можем использовать ряд предлагаемых пакетом функций:

func Create(name string) (*File, error) // создание файла с именем name

func Open(name string) (*File, error) // открытие файла с именем name
Данный объект имеет ряд методов, позволяющих вернуть имя файла, изменить права доступа или владельца файла. Нас же, прежде всего, будет интересовать метод Close(), который должен быть вызван при закрытии файла, чтобы освободить занятые нашей программой ресурсы операционной системы. Чтобы быть уверенными, что файл будет закрыт, а ресурсы освобождены даже в случае критической непредвиденной ошибки, рекомендуется сразу после проверки, что файл создан / открыт запланировать его закрытие с помощью оператора отложенного вызова defer:

f, err := os.Open("fileName")
if err != nil {
	...
}
defer f.Close()
С имеющимися методами данного типа вы можете ознакомиться в документации, примеры же работы с этим объектом мы приведем в следующем шаге, когда рассмотрим более удобные способы работы с этим типом.

Переименование и удаление файлов делается через функции Rename и Remove:

// создаем файл
os.Create("text.txt")
// переименовываем файл
os.Rename("text.txt", "new_text.txt")
// удаляем файл
os.Remove("new_text.txt")
// кстати, os позволяет работать не только с файлами
// выходим из программы:
os.Exit(0)
Так же мы можем получать информацию файлов и сравнивать их:

file1, _ := os.Create("text.txt")
file2, _ := os.Create("text.txt")
info1, _ := file1.Stat() // функция Stat возвращает информацию о файле и ошибку
info2, _ := file2.Stat()
fmt.Println(os.SameFile(info1, info2)) // true

// вот что мы можем получить из FileInfo:
// A FileInfo describes a file and is returned by Stat and Lstat.
type FileInfo interface {
	Name() string       // base name of the file
	Size() int64        // length in bytes for regular files; system-dependent for others
	Mode() FileMode     // file mode bits
	ModTime() time.Time // modification time
	IsDir() bool        // abbreviation for Mode().IsDir()
	Sys() interface{}   // underlying data source (can return nil)
}
Также существует удобная функция WriteString у файлов которая записывает строки в конце файла. Например, нужно открыть существующий файл с какой-то информацией и записать, не затрагивая старую.

file1, _ := os.Create("text.txt")
file1.WriteString("1 строка \n")
file1.WriteString("2 строка \n")
file1.Close()

// внутри файла будет:
// 1 строка 
// 2 строка 

```

### bufio

```Go

Пакет bufio предоставляет нам ряд инструментов для удобного чтения и записи информации из объектов, удовлетворяющих интерфейсам io.Reader и io.Writer. Нас в этом вопросе прежде всего будут интересовать типы bufio.Reader, bufio.Writer и bufio.Scanner. Но перед тем как перейти к рассмотрению данных типов сделаем одно замечание (мы уже говорили об этом раньше в разделе об интерфейсах): объекты в пакете bufio имеют собственный буфер, позволяющий им реализовать дополнительные методы по сравнению с интерфейсами io.Reader и io.Writer, это необходимо учитывать в работе. Если вы не помните, о чем шла речь, вернитесь ненадолго к приведенному в том разделе примеру.

bufio.Reader
Данный тип создается с помощью функций:

func NewReader(rd io.Reader) *Reader // создает Reader со стандартным буфером 4096 байт

func NewReaderSize(rd io.Reader, size int) *Reader // создает Reader с произвольным буфером
Рассмотрим некоторые из методов bufio.Reader и примеры работы:

file, err := os.Open("test.txt")
if err != nil {
	...
}
defer file.Close()

rd := bufio.NewReader(file)

buf := make([]byte, 10)
n, err := rd.Read(buf) // читаем в buf 10 байт из ранее открытого файла
if err != nil && err != io.EOF {
	// io.EOF не совсем ошибка - это состояние, указывающее, что файл прочитан до конца
	...
}
fmt.Printf("прочитано %d байт: %s\n", n, buf) // прочитано 10 байт: bufio ...

s, err := rd.ReadString('\n') // читаем данные до разрыва абзаца ('\n')
fmt.Printf("%s\n", s)         // ... здесь будет строка
bufio.Reader позволяет читать данные по байтам, рунам, строкам и пр., указывать символ, на котором необходимо прекратить чтение. Когда данные будут прочитаны до конца, метод вернет ошибку io.EOF.

bufio.Writer
bufio.Writer создан для записи в объекты, удовлетворяющие интерфейсу io.Writer, но предоставляет ряд более высокоуровневых методов, в частности метод WriteString(s string):

file, err := os.Create("test.txt")
if err != nil {
	...
}
defer file.Close()

w := bufio.NewWriter(file)
n, err := w.WriteString("Запишем строку")
if err != nil {
	...
}
fmt.Printf("Записано %d байт\n", n) // Записано 27 байт

// bufio.Writer имеет собственный буфер, чтобы быть уверенным, что данные точно записаны,
// вызываем метод Flush()
w.Flush()
Как вы уже поняли, создается объект функцией NewWriter(w io.Writer).

bufio.Scanner
bufio.Scanner создан для построчного чтения данных. Создается он функцией NewScanner(r io.Reader), посмотрим, как работает этот тип:

file, err := os.Open("test.txt")
if err != nil {
	panic(err)
}
defer file.Close()

s := bufio.NewScanner(file)

// Я заранее записал в файл 5 цифр, каждую на новой строке
for s.Scan() { // возвращает true, пока файл не будет прочитан до конца
	fmt.Printf("%s\n", s.Text()) // s.Text() содержит данные, считанные на данной итерации
}

// 1
// 2
// 3
// 4
// 5

```

```
В каких случаях использовать тот или иной пакет стандартной библиотеки
В качестве небольшого отступления рассмотрим вопрос о целесообразности использования того или иного метода чтения / записи. Если объем данных небольшой, то разумно использовать функции из пакета io/ioutil - они позволяют нам не заботиться о закрытии файла (кроме функции ReadAll, которая в качестве аргумента получает тип io.Reader), кроме того, все данные считываются / записываются за раз. Стоит дополнить, что многие методы io/ioutil это просто удобные обертки (абстракции) под капотом которых тот же os.OpenFile и тд. Поэтому если хочется более близкий доступ к системе и возможностям используйте os.

Но что если объем данных велик, или не все данные используются единовременно (или готовятся к записи постепенно)? Тогда правильно будет отдать предпочтение пакету bufio, который может считывать данные поэтапно, в т.ч. построчно, записывать данные постепенно.
```

### path и path/filepath
```Go
В завершении мы кратко остановимся на двух пакетах, реализующих функционал создания и обработки путей к файлам path и path/filepath.

Пакет path использует в качестве разделителя символ "/" (слэш), как следствие, он может использоваться только в операционных системах с соответствующим разделителем - unix системах (MaсOS, GNU/Linux и другие). Второй же пакет более универсален и выбирает разделитель в зависимости от операционной системы: для Windows используется обратный слэш - "\".

Напечатаем все файлы в директории

package main

import (
	"fmt"
	"os"
	"path/filepath"
)

func walkFunc(path string, info os.FileInfo, err error) error {
   if err != nil {
      return err // Если по какой-то причине мы получили ошибку, проигнорируем эту итерацию
   }

   // Проигнорируем директории
   if info.IsDir() {
      // Не спускаемся в директории .git и .idea
      if info.Name() == ".git" || info.Name() == ".idea" {
         return filepath.SkipDir
      }

      return nil
   }

   fmt.Printf("Name: %s\tSize: %d byte\tPath: %s\n", info.Name(), info.Size(), path)
   return nil
}

func main() {
	const root = "./test" // Файлы моей программы находятся в другой директории

	if err := filepath.Walk(root, walkFunc); err != nil {
		fmt.Printf("Какая-то ошибка: %v\n", err)
	}

	// Name: file1     Size: 6 byte    Path: test/dir1/file1
	// Name: file2     Size: 6 byte    Path: test/dir1/file2
	// Name: file3     Size: 6 byte    Path: test/dir2/file3
	// Name: file4     Size: 6 byte    Path: test/dir3/file4
	// Name: file5     Size: 6 byte    Path: test/dir3/file5
	// Name: file6     Size: 6 byte    Path: test/dir3/file6
}
```

печать содержимого всех файлов в архиве и в его поддиректориях

```Go

package main

import (
	"encoding/csv"
	"fmt"
	"archive/zip"
)


func main() {
	arch, _ := zip.OpenReader("task.zip")
	defer arch.Close()

	for _, f := range arch.File {
		file, _ := f.Open()
		temp, _ := csv.NewReader(file).ReadAll()
		fmt.Println(temp)
        } 
	}

```